Fig generator for each graphs class

In [2]:
import sys
from pathlib import Path

sys.path.append(str(Path("..").resolve()))

from objects import *

output_dir = Path.cwd().parent.parent / "docs/figs"
output_dir.mkdir(parents=True, exist_ok=True)
max_nodes = 10


In [3]:
for num_u in range(1, max_nodes):
    for num_v in range(num_u, max_nodes - num_u + 1):
        for num_edges in range(num_u + num_v - 1, num_u * num_v + 1):
            try:
                g = BipartiteGraph(num_u, num_v, num_edges)
            except RuntimeError:
                continue

            file_path = output_dir / "bipartite" / f"{g.label}.png"
            g.plot(filename=str(file_path))


In [4]:
for n in range(4, max_nodes + 1, 2):
    try:
        g = CubicGraphs(n)
    except RuntimeError:
        continue

    for i, graph in enumerate(g.graphs, start=0):
        file_path = output_dir / "cubic" / f"cubic_{n}_nodes_{i}.png"
        g.plot(index=i, filename=str(file_path))


In [5]:
for n in range(1, 8):
    try:
        g = TriangleFreeGraphs(n)
    except RuntimeError:
        continue

    for i, graph in enumerate(g.graphs, start=1):
        deg_seq = g.degree_sequences[i-1]
        deg_str = "_".join(map(str, deg_seq))
        file_name = f"tf_{deg_str}.png"

        file_path = output_dir / "triangle-free" / file_name
        g.plot(index=i-1, filename=str(file_path))


Dataset construction


In [ ]:
import numpy as np
import networkx as nx
import pandas as pd
from sage.all import *
from utils import gpn

df_data = []
for n in range(1, 10):
    print(n)
    for g in graphs.nauty_geng(f"{n} -c"):
        g = g.networkx_graph()

        if not nx.is_connected(g):
            print("g is not connected")

        num_nodes = g.number_of_nodes()
        num_edges = g.number_of_edges()
        degrees = [d for _, d in g.degree()]
        max_degree = max(degrees)
        min_degree = min(degrees)
        avg_degree = np.mean(degrees)
        median_degree = np.median(degrees)
        longest_path = nx.diameter(g)

        adj_matrix = nx.to_numpy_array(g)
        largest_eigenvalue = max(np.linalg.eigvals(adj_matrix)).real

        is_regular = all(d == degrees[0] for d in degrees)
        is_tree = nx.is_tree(g)
        is_planar = nx.check_planarity(g)[0]
        is_bipartite = nx.is_bipartite(g)
        is_cubic = all(d == 3 for d in degrees)

        gpn_value = gpn(g)

        label = f"g{num_nodes}_{num_edges}_{degrees}"

        df_data.append(
            [
                label,
                num_nodes,
                num_edges,
                max_degree,
                min_degree,
                avg_degree,
                median_degree,
                longest_path,
                largest_eigenvalue,
                is_regular,
                is_tree,
                is_planar,
                is_bipartite,
                is_cubic,
                gpn_value,
            ]
        )

columns = [
    "label",
    "num_nodes",
    "num_edges",
    "max_degree",
    "min_degree",
    "avg_degree",
    "median_degree",
    "longest_path",
    "largest_eigenvalue",
    "is_regular",
    "is_tree",
    "is_planar",
    "is_bipartite",
    "is_cubic",
    "gpn_num",
]

df = pd.DataFrame(df_data, columns=columns)
df.to_csv("../../data/all_graphs_9n.csv", index=False)


1
2
3
4
5
6
7
8
9
